<h1 id="tocheading">Table of Contents</h1>
<div id="toc"></div>

In [630]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

### import libraries

In [1]:
import requests
import re
from bs4 import BeautifulSoup

### example on how to find links in a page

In [8]:
link = 'https://www.analyticsvidhya.com/blog/2016/02/complete-guide-parameter-tuning-gradient-boosting-gbm-python/'

In [9]:
r = requests.get(link)
cont = r.content
soup = BeautifulSoup(cont,'lxml')
# soup.prettify()

In [ ]:
txt = 'https://www.analyticsvidhya.com/blog/2016/02/complete-guide-parameter-tuning-gradient-boosting-gbm-python/'

In [24]:
pattern = r'(https://www.analyticsvidhya.com/blog/201[0-9]/0?[0-9]{2})(/[a-zA-z-]*)(/.*)'

In [90]:
matchList = re.findall(pattern, txt)

In [94]:
matchList[0][2]

'/#comment-152592'

In [107]:
internal_links = []
for x in soup.find_all('a',href=re.compile(pattern)):
    inurl = x.get('href')
    matchlist = re.findall(pattern, inurl)
    if len(matchlist[0][2])<2:
        internal_links.append(inurl)

In [112]:
list(set(internal_links))

['https://www.analyticsvidhya.com/blog/2016/02/business-analytics-bml-munjal-university/',
 'https://www.analyticsvidhya.com/blog/2015/11/quick-introduction-boosting-algorithms-machine-learning/',
 'https://www.analyticsvidhya.com/blog/2017/09/understaing-support-vector-machine-example-code/',
 'https://www.analyticsvidhya.com/blog/2017/09/common-machine-learning-algorithms/',
 'https://www.analyticsvidhya.com/blog/2018/03/introduction-k-neighbours-algorithm-clustering/',
 'https://www.analyticsvidhya.com/blog/2017/09/naive-bayes-explained/',
 'https://www.analyticsvidhya.com/blog/2015/05/data-visualization-python/',
 'https://www.analyticsvidhya.com/blog/2016/08/winners-approach-smart-recruits/',
 'https://www.analyticsvidhya.com/blog/2015/08/comprehensive-guide-regression/',
 'https://www.analyticsvidhya.com/blog/2018/01/online-learning-guide-text-classification-vowpal-wabbit-vw/',
 'https://www.analyticsvidhya.com/blog/2016/02/time-series-forecasting-codes-python/',
 'https://www.an

### define functions to parse

In [15]:
def url_to_soup(link):
    try:
        r = requests.get(link)
        soup = BeautifulSoup(r.content,'lxml')
    except Exception as e:
        print(e)
    else:
        return soup

In [16]:
def get_title(soup):
    try:
        title = soup.title.text.strip()
    except Exception as e:
        print(e)
    else:
        return title

In [17]:
def get_body(soup):
    try:
        templist = []
        for x in soup.find_all(['h1','h2','h3','p']):
            templist.append(x.text)
            body = ' '.join(templist).replace("\n"," ").replace("\r"," ").replace("\xa0"," ")
    except Exception as e:
        print(e)
    else:
        return body

In [18]:
def get_links(soup,pattern):
    try:
        internal_links = []
        for x in soup.find_all('a',href=re.compile(pattern)):
            inurl = x.get('href')
            matchlist = re.findall(pattern, inurl)
            if len(matchlist[0][2])<2:
                internal_links.append(inurl)
    except Exception as e:
        print(e)
    else:
        return list(set(internal_links))

In [481]:
def parse(link,link_pattern):
    """
    given a url, extract hotlinks matching a link_pattern ~ define using Regex
    """
    try:
        url    = link
        mysoup = url_to_soup(link)
        title  = get_title(mysoup)
        body   = get_body(mysoup)
        links  = get_links(mysoup,pattern)
    except Exception as e:
        print(e)
    else:
        return [link,title,body,links]

In [20]:
def update_scaper_dict(parsed,scraper_dict):
    try:
        scraper_dict['url'].append(parsed[0])
        scraper_dict['title'].append(parsed[1])
        scraper_dict['body'].append(parsed[2])
        scraper_dict['links'].append(parsed[3])
    except Exception as e:
        print(e)
    else:
        return scraper_dict

### test parsing functions

In [21]:
scraper_dict = {'url':[],
                'title':[],
                'body':[],
                'links':[]}

In [22]:
link1 = 'https://www.analyticsvidhya.com/blog/2017/08/catboost-automated-categorical-data/'
link2 = "https://www.analyticsvidhya.com/blog/2017/06/which-algorithm-takes-the-crown-light-gbm-vs-xgboost/"
link3 = "https://www.analyticsvidhya.com/blog/2017/09/understaing-support-vector-machine-example-code/"

In [24]:
pattern = r'(https://www.analyticsvidhya.com/blog/201[0-9]/0?[0-9]{2})(/[a-zA-z-]*)(/.*)'

In [31]:
results = parse(link3,pattern)

In [32]:
updt_dict = update_scaper_dict(results,scraper_dict)

### create a class to handle parsed data

In [628]:
class superdict():
    
    def __init__(self):
        self.dict = {'url':[],'title':[],'body':[],'links':[]}
        self.expanded_list = []
        self.unvisited     = []
        self.top_urls      = []
        
    def update_with(self,parsed_obj):
        try:
            self.dict['url'].append(parsed_obj[0])
            self.dict['title'].append(parsed_obj[1])
            self.dict['body'].append(parsed_obj[2])
            self.dict['links'].append(parsed_obj[3])
        except Exception as e:
            print(e)

    def get_unnest(self):
        try:
            self.expanded_list = sorted([item for nestedlist in self.dict['links'] for item in nestedlist])
        except Exception as e:
            print(e)

    def get_not_visited(self):
        try:
            l1 = set(self.expanded_list)
            l2 = set(self.dict['url'])
            self.unvisited = sorted(list(l1.difference(l2)))
        except Exception as e:
            print(e)
            
    def get_choice(self):
        try:
            temp = random.choice(self.unvisited) 
        except Exception as e:
            print(e)
        else:
            return temp
        
    def wrapper(self):
        try:
            self.get_unnest()
            self.get_not_visited()
        except Exception as e:
            print(e)
        else:
            return self.get_choice()
        

### automate parsing process

In [610]:
b = superdict()
i=1
link = 'https://www.analyticsvidhya.com/blog/2018/12/guide-digital-marketing-analytics/'
pattern = r'(https://www.analyticsvidhya.com/blog/201[0-9]/0?[0-9]{2})(/[a-zA-z-]*)(/.*)'
condition = True

In [611]:
import time
start = time.time()
while i <=10000 and condition:
    print(link)
    parsedObj = parse(link,pattern)
    b.update_with(parsedObj)
    link = b.wrapper()
    
    i+=1
    if len(b.unvisited) == 0:
        condition = False
    time.sleep(5)

end = time.time()-start
end

https://www.analyticsvidhya.com/blog/2018/12/guide-digital-marketing-analytics/
https://www.analyticsvidhya.com/blog/2014/12/framework-kaggle-competition/
https://www.analyticsvidhya.com/blog/2017/09/naive-bayes-explained/
https://www.analyticsvidhya.com/blog/2013/07/advanced-analytics-certifications/
https://www.analyticsvidhya.com/blog/2013/11/interview-top-data-scientist-kaggler-mr-steve-donoho/
https://www.analyticsvidhya.com/blog/2018/03/pandas-on-ray-python-library-make-processing-faster/
https://www.analyticsvidhya.com/blog/2018/02/rllib-library-aims-to-make-reinforcement-learning-easier/
https://www.analyticsvidhya.com/blog/2018/03/build-your-own-facial-emoji-using-deep-learning/
https://www.analyticsvidhya.com/blog/2015/01/decision-tree-simplified/
https://www.analyticsvidhya.com/blog/2017/06/course-review-pg-diploma-in-data-analytics-by-upgrad-iiit-b/
https://www.analyticsvidhya.com/blog/2016/04/complete-tutorial-tree-based-modeling-scratch-in-python/
https://www.analyticsvid

https://www.analyticsvidhya.com/blog/2018/01/online-learning-guide-text-classification-vowpal-wabbit-vw/
https://www.analyticsvidhya.com/blog/2016/06/bayesian-statistics-beginners-simple-english/
https://www.analyticsvidhya.com/blog/2016/02/guide-build-predictive-models-segmentation/
https://www.analyticsvidhya.com/blog/2016/04/deep-learning-computer-vision-introduction-convolution-neural-networks/
https://www.analyticsvidhya.com/blog/2013/07/read-books-analysts-or-interested-analytics/
https://www.analyticsvidhya.com/blog/2018/07/learn-and-test-your-machine-learning-skills-with-avs-new-practice-problems-and-free-courses/
https://www.analyticsvidhya.com/blog/2015/10/inferential-descriptive-statistics-beginners-r/
https://www.analyticsvidhya.com/blog/2018/02/google-uses-ai-for-filtering-android-apps/
https://www.analyticsvidhya.com/blog/2014/08/big-data-big-difference/
https://www.analyticsvidhya.com/blog/2016/09/most-active-data-scientists-free-books-notebooks-tutorials-on-github/
http

https://www.analyticsvidhya.com/blog/2014/01/tips-creating-winning-dashboard/
https://www.analyticsvidhya.com/blog/2018/03/woebot-mental-health-chatbot-supported-andrew-ng/
https://www.analyticsvidhya.com/blog/2016/01/guide-data-exploration/
https://www.analyticsvidhya.com/blog/2018/04/introductory-guide-understand-how-anns-conceptualize-new-ideas/
https://www.analyticsvidhya.com/blog/2018/03/essentials-of-deep-learning-sequence-to-sequence-modelling-with-attention-part-i/
https://www.analyticsvidhya.com/blog/2018/05/data-analytics-in-the-indian-agriculture-industry/
https://www.analyticsvidhya.com/blog/2018/02/the-different-methods-deal-text-data-predictive-python/
https://www.analyticsvidhya.com/blog/2015/07/dimension-reduction-methods/
https://www.analyticsvidhya.com/blog/2016/03/winning-solutions-dyd-competition-xgboost-ruled/
https://www.analyticsvidhya.com/blog/2016/07/deeper-regression-analysis-assumptions-plots-solutions/
https://www.analyticsvidhya.com/blog/2017/12/fundamental

https://www.analyticsvidhya.com/blog/2017/11/information-retrieval-using-kdtree/
https://www.analyticsvidhya.com/blog/2017/03/read-commonly-used-formats-using-python/
https://www.analyticsvidhya.com/blog/2018/03/tableau-for-advanced-users-easy-expertise-in-data-visualisation/
https://www.analyticsvidhya.com/blog/2015/08/interactive-data-visualization-library-python-bokeh/
https://www.analyticsvidhya.com/blog/2017/11/flashtext-a-library-faster-than-regular-expressions/
https://www.analyticsvidhya.com/blog/2015/06/infographic-cheat-sheet-data-exploration-python/
https://www.analyticsvidhya.com/blog/2016/03/review-big-data-hadoop-developer-certification-simplilearn/
https://www.analyticsvidhya.com/blog/2013/08/importance-segmentation-create/
https://www.analyticsvidhya.com/blog/2018/01/kolkata-analytics-google-maps-traffic/
https://www.analyticsvidhya.com/blog/2016/08/innovation-in-analytics-education-great-lakes-using-mentored-learning-for-online-courses/
https://www.analyticsvidhya.com/

https://www.analyticsvidhya.com/blog/2018/10/predicting-stock-price-machine-learningnd-deep-learning-techniques-python/
https://www.analyticsvidhya.com/blog/2015/10/tips-tricks-awesom-data-science-jobs/
https://www.analyticsvidhya.com/blog/2016/08/solutions-for-skilltest-in-statistics-revealed/
https://www.analyticsvidhya.com/blog/2016/03/practical-guide-principal-component-analysis-python/
https://www.analyticsvidhya.com/blog/2018/11/tutorial-text-classification-ulmfit-fastai-library/
https://www.analyticsvidhya.com/blog/2018/04/nvidias-deep-learning-model-reconstruct-incomplete-photos-incredible-results/
https://www.analyticsvidhya.com/blog/2014/11/hadoop-mapreduce/
https://www.analyticsvidhya.com/blog/2015/09/startups-analytics-data-science-closer/
https://www.analyticsvidhya.com/blog/2017/07/debugging-neural-network-with-tensorboard/
https://www.analyticsvidhya.com/blog/2015/12/special-year-resolutions-data-scientist/
https://www.analyticsvidhya.com/blog/2015/09/selection-technique

...

https://www.analyticsvidhya.com/blog/2018/03/google-making-music-help-machine-learning/
https://www.analyticsvidhya.com/blog/2018/10/computer-vision-approach-hand-gesture-recognition/
https://www.analyticsvidhya.com/blog/2018/02/boston-startup-optimization-analytics-machine-learning/
https://www.analyticsvidhya.com/blog/2014/05/build-word-cloud-text-mining-tools/
https://www.analyticsvidhya.com/blog/2015/07/top-youtube-videos-machine-learning-neural-network-deep-learning/
https://www.analyticsvidhya.com/blog/2018/03/google-deep-learning-make-computers-better-with-age/
https://www.analyticsvidhya.com/blog/2014/11/step-world-sas-iml/
https://www.analyticsvidhya.com/blog/2018/04/stanford-nlp-course-projects-available-online-theyre-super-impressive/
https://www.analyticsvidhya.com/blog/2015/02/massive-open-online-courses-moocs/
https://www.analyticsvidhya.com/blog/2018/09/best-ted-talks-artificial-intelligence-must-watch/
https://www.analyticsvidhya.com/blog/2015/07/guide-data-visualizatio

https://www.analyticsvidhya.com/blog/2013/07/analytics-rockstar/
https://www.analyticsvidhya.com/blog/2018/08/machine-learning-algorithm-fix-wikis/
https://www.analyticsvidhya.com/blog/2018/08/perform-automated-time-series-modeling-datarobots/
https://www.analyticsvidhya.com/blog/2018/09/speech-object-recognition-one-model-mit-ml/
https://www.analyticsvidhya.com/blog/2013/05/creating-drilldowns-visual-cues-in-qlikview/
https://www.analyticsvidhya.com/blog/2015/10/read-books-for-beginners-machine-learning-artificial-intelligence/
https://www.analyticsvidhya.com/blog/2014/09/data-business-analyst-mpaani-mumbai/
https://www.analyticsvidhya.com/blog/2015/09/infographic-tools-data-visualization/
https://www.analyticsvidhya.com/blog/2018/06/china-school-facial-recognition-analyse-students/
https://www.analyticsvidhya.com/blog/2018/03/machine-learning-can-help-you-draw/
https://www.analyticsvidhya.com/blog/2014/06/leader-marketing-mastercard-gurgaon/
https://www.analyticsvidhya.com/blog/2017/

https://www.analyticsvidhya.com/blog/2017/10/svm-skilltest/
https://www.analyticsvidhya.com/blog/2015/12/improve-machine-learning-results/
https://www.analyticsvidhya.com/blog/2018/02/jupyterlab-available-for-general-public/
https://www.analyticsvidhya.com/blog/2018/03/sas-built-machine-learning-model-rank-the-best-places-world/
https://www.analyticsvidhya.com/blog/2013/05/good-analyst-vs-bad-analyst/
https://www.analyticsvidhya.com/blog/2015/03/text-mining-subject-extraction-google-api/
https://www.analyticsvidhya.com/blog/2014/04/survival-analysis-model-you/
https://www.analyticsvidhya.com/blog/2018/09/deep-learning-video-classification-python/
https://www.analyticsvidhya.com/blog/2015/09/applications-data-science/
https://www.analyticsvidhya.com/blog/2017/09/building-machine-learning-model-fun-using-orange/
https://www.analyticsvidhya.com/blog/2016/06/quick-guide-build-recommendation-engine-python/
https://www.analyticsvidhya.com/blog/2015/03/data-science-trainings-failures-overcome

https://www.analyticsvidhya.com/blog/2014/05/hadoop-simplified/
https://www.analyticsvidhya.com/blog/2015/07/overview-analytics-industry-india/
https://www.analyticsvidhya.com/blog/2018/07/iit-roorkee-researchers-using-computer-vision-monitor-railway-tracks/
https://www.analyticsvidhya.com/blog/2018/05/ibm-platform-combines-data-science-data-engineering-one-powerful-package/
https://www.analyticsvidhya.com/blog/2018/07/google-brain-researchers-unveil-ml-model-that-manipulates-neural-networks/
https://www.analyticsvidhya.com/blog/2018/05/deep-learning-faq/
https://www.analyticsvidhya.com/blog/2018/09/datahack-radio-lyft-dr-alok-gupta/
https://www.analyticsvidhya.com/blog/2015/02/avoid-over-fitting-regularization/
https://www.analyticsvidhya.com/blog/2018/04/this-ai-create-cartoons-text-description/
https://www.analyticsvidhya.com/blog/2018/04/this-deep-learning-algorithm-detects-face-swaps-videos/
https://www.analyticsvidhya.com/blog/2014/11/cognos-developer-hansa-cequity-india/
https:/

5692.741294622421

### persistence

In [623]:
import pandas as pd
from pandas import DataFrame, Series
df = DataFrame(b.dict)

In [624]:
df_to_save = df.iloc[:,0:3]

In [625]:
import sqlite3
db = sqlite3.connect("scraperdb.sqlite")
df_to_save.to_sql("vidhya",db,index=False,if_exists="append")

In [626]:
import pickle
with open('vidhya2.pkl', 'wb') as handle:
    pickle.dump(b.dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('vidhya.pickle', 'rb') as handle:
#     b = pickle.load(handle)